In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

def display_images(images):
    grid = utils.make_grid(images)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.show()

def format_data_for_display(tensor):
    maxVal = tensor.max()
    minVal = abs(tensor.min())
    maxVal = max(maxVal,minVal)
    output_data = tensor / maxVal
    output_data = output_data / 2
    output_data = output_data + 0.5
    return output_data

In [2]:
from __future__ import print_function
import zipfile
import os
import pdb
import torch
import h5py
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, transforms, utils

import numpy as np
import torch.nn as nn
import torch

output_height=64
output_width=64

class TransposeDepthInput(object):
    def __call__(self, depth):
        depth = depth.transpose((2, 0, 1))
        depth = torch.from_numpy(depth)
        depth = depth.unsqueeze(0)
        depth = nn.functional.interpolate(depth, size=(output_height, output_width), mode='bilinear', align_corners=False)
        depth = torch.log(depth[0])
        return depth

rgb_data_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
])

depth_data_transforms = transforms.Compose([
    TransposeDepthInput(),
])

input_for_plot_transforms = transforms.Compose([
    transforms.Resize((output_height, output_width)),    # Different for Input Image & Depth Image
    transforms.ToTensor(),
])

class NYUDataset(Dataset):
    def __init__(self, filename, type, rgb_transform = None, depth_transform = None):
        f = h5py.File(filename, 'r')
        if type == "training":
            self.images = f['images'][0:1024]
            self.depths = f['depths'][0:1024]
        elif type == "validation":
            self.images = f['images'][1024:1248]
            self.depths = f['depths'][1024:1248]
        elif type == "test":
            self.images = f['images'][1248:]
            self.depths = f['depths'][1248:]
        self.rgb_transform = rgb_transform
        self.depth_transform = depth_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = image.transpose((2, 1, 0))
        image = Image.fromarray(image)
        if self.rgb_transform:
            image = self.rgb_transform(image)
        depth = self.depths[idx]
        depth = np.reshape(depth, (1, depth.shape[0], depth.shape[1]))
        depth = depth.transpose((2, 1, 0))
        if self.depth_transform:
            depth = self.depth_transform(depth)
        sample = {'image': image, 'depth': depth}
        return sample

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=3, padding=1, stride=2):
        super(UNetConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_size, out_size, kernel_size=kernel_size, padding=padding, stride=stride)

    def forward(self, x):
        return self.conv(x)

class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=2, stride=2, space_dropout=False):
        super(UNetUpBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=kernel_size, stride=stride)

    def forward(self, x):
        return self.up(x)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.activation = F.relu

        self.conv_block3_16 = UNetConvBlock(3, 16)
        self.conv_block16_32 = UNetConvBlock(16, 32)
        self.conv_block32_64 = UNetConvBlock(32, 64)

        self.up_block64_32 = UNetUpBlock(64, 32)
        self.up_block32_16 = UNetUpBlock(32, 16)
        self.up_block16_1 = UNetUpBlock(16, 1)

    def forward(self, x):
        block1 = self.activation(self.conv_block3_16(x))
        block2 = self.activation(self.conv_block16_32(block1))
        block3 = self.activation(self.conv_block32_64(block2))
        up1 = self.activation(self.up_block64_32(block3))
        up2 = self.activation(self.up_block32_16(up1))
        up3 = self.up_block16_1(up2)
        return up3


In [4]:
model = UNet()
from torchsummary import summary
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
     UNetConvBlock-2           [-1, 16, 32, 32]               0
            Conv2d-3           [-1, 32, 16, 16]           4,640
     UNetConvBlock-4           [-1, 32, 16, 16]               0
            Conv2d-5             [-1, 64, 8, 8]          18,496
     UNetConvBlock-6             [-1, 64, 8, 8]               0
   ConvTranspose2d-7           [-1, 32, 16, 16]           8,224
       UNetUpBlock-8           [-1, 32, 16, 16]               0
   ConvTranspose2d-9           [-1, 16, 32, 32]           2,064
      UNetUpBlock-10           [-1, 16, 32, 32]               0
  ConvTranspose2d-11            [-1, 1, 64, 64]              65
      UNetUpBlock-12            [-1, 1, 64, 64]               0
Total params: 33,937
Trainable params: 33,937
Non-trainable params: 0
---------------------------------

In [5]:
import torch
from torchvision import datasets, transforms, utils

batch_size = 1

train_loader = torch.utils.data.DataLoader(NYUDataset( 'nyu_depth_v2_labeled.mat', 
                                                       'training', 
                                                        rgb_transform = rgb_data_transforms, 
                                                        depth_transform = depth_data_transforms), 
                                                        batch_size = batch_size, 
                                                        shuffle = False, num_workers = 0)

val_loader = torch.utils.data.DataLoader(NYUDataset( 'nyu_depth_v2_labeled.mat',
                                                     'validation', 
                                                     rgb_transform = rgb_data_transforms, 
                                                     depth_transform = depth_data_transforms), 
                                                     batch_size = batch_size, 
                                                     shuffle = False, num_workers = 0)

In [8]:
# from torchvision.utils import save_image
# from IPython.core.display import Image, display

data = next(iter(train_loader))
first_rgb = data['image']
first_depth = data['depth']
print(first_rgb.type())
print(first_rgb.shape)

display_images(first_rgb)
# display_images(format_data_for_display(first_depth))
# save_image(first_rgb, 'real_image.png')

# Image('real_image.png', width=500)

torch.FloatTensor
torch.Size([1, 3, 64, 64])


In [8]:
from __future__ import print_function
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils

from torch.autograd import Variable
from logger import Logger
import pdb
import os
import re
import numpy as np

epochs = 1000
log_interval = 10
model_folder = 'small-unet'

model = UNet()
loss_function = F.mse_loss
optimizer = optim.Adam(model.parameters(), amsgrad=True, lr=0.001)
logger = Logger('./logs/' + model_folder)

def train_Unet(epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        rgb, depth = data['image'], data['depth']
        optimizer.zero_grad()
        output = model(rgb)
        target = depth[:,0,:,:].view(list(depth.shape)[0], 1, output_height, output_width)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(rgb), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        if batch_idx == 0: break

def validate_Unet():
    print('validating unet')
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(val_loader):
            rgb, depth = data['image'], data['depth']
            output = model(rgb)
            target = depth[:,0,:,:].view(list(depth.shape)[0], 1, output_height, output_width)
            validation_loss += F.mse_loss(output, target)
        validation_loss /= batch_idx
        print('\nValidation set: Average loss: {:.6f} {:.6f} {:.6f}\n'.format(validation_loss))

folder_name = "models/" + model_folder
if not os.path.exists(folder_name): os.mkdir(folder_name)

for epoch in range(1, epochs + 1):
    print("********* Training the Unet Model **************")
    train_Unet(epoch)
    if epoch % 25== 0:
        model_file = folder_name + "/" + 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
#    validate_Unet()

********* Training the Unet Model **************
Train Epoch: 1 [0/1024 (0%)]	Loss: 0.967025
********* Training the Unet Model **************
Train Epoch: 2 [0/1024 (0%)]	Loss: 0.957579
********* Training the Unet Model **************
Train Epoch: 3 [0/1024 (0%)]	Loss: 0.947753
********* Training the Unet Model **************
Train Epoch: 4 [0/1024 (0%)]	Loss: 0.937050
********* Training the Unet Model **************
Train Epoch: 5 [0/1024 (0%)]	Loss: 0.925226
********* Training the Unet Model **************
Train Epoch: 6 [0/1024 (0%)]	Loss: 0.911859
********* Training the Unet Model **************
Train Epoch: 7 [0/1024 (0%)]	Loss: 0.896216
********* Training the Unet Model **************
Train Epoch: 8 [0/1024 (0%)]	Loss: 0.877712
********* Training the Unet Model **************
Train Epoch: 9 [0/1024 (0%)]	Loss: 0.855017
********* Training the Unet Model **************
Train Epoch: 10 [0/1024 (0%)]	Loss: 0.826541
********* Training the Unet Model **************
Train Epoch: 11 [0/1

Train Epoch: 101 [0/1024 (0%)]	Loss: 0.019250
********* Training the Unet Model **************
Train Epoch: 102 [0/1024 (0%)]	Loss: 0.018962
********* Training the Unet Model **************
Train Epoch: 103 [0/1024 (0%)]	Loss: 0.018680
********* Training the Unet Model **************
Train Epoch: 104 [0/1024 (0%)]	Loss: 0.018409
********* Training the Unet Model **************
Train Epoch: 105 [0/1024 (0%)]	Loss: 0.018153
********* Training the Unet Model **************
Train Epoch: 106 [0/1024 (0%)]	Loss: 0.017905
********* Training the Unet Model **************
Train Epoch: 107 [0/1024 (0%)]	Loss: 0.017663
********* Training the Unet Model **************
Train Epoch: 108 [0/1024 (0%)]	Loss: 0.017440
********* Training the Unet Model **************
Train Epoch: 109 [0/1024 (0%)]	Loss: 0.017232
********* Training the Unet Model **************
Train Epoch: 110 [0/1024 (0%)]	Loss: 0.017029
********* Training the Unet Model **************
Train Epoch: 111 [0/1024 (0%)]	Loss: 0.016836
****

Train Epoch: 188 [0/1024 (0%)]	Loss: 0.010252
********* Training the Unet Model **************
Train Epoch: 189 [0/1024 (0%)]	Loss: 0.010206
********* Training the Unet Model **************
Train Epoch: 190 [0/1024 (0%)]	Loss: 0.010161
********* Training the Unet Model **************
Train Epoch: 191 [0/1024 (0%)]	Loss: 0.010116
********* Training the Unet Model **************
Train Epoch: 192 [0/1024 (0%)]	Loss: 0.010071
********* Training the Unet Model **************
Train Epoch: 193 [0/1024 (0%)]	Loss: 0.010026
********* Training the Unet Model **************
Train Epoch: 194 [0/1024 (0%)]	Loss: 0.009982
********* Training the Unet Model **************
Train Epoch: 195 [0/1024 (0%)]	Loss: 0.009938
********* Training the Unet Model **************
Train Epoch: 196 [0/1024 (0%)]	Loss: 0.009894
********* Training the Unet Model **************
Train Epoch: 197 [0/1024 (0%)]	Loss: 0.009850
********* Training the Unet Model **************
Train Epoch: 198 [0/1024 (0%)]	Loss: 0.009807
****

Train Epoch: 287 [0/1024 (0%)]	Loss: 0.006469
********* Training the Unet Model **************
Train Epoch: 288 [0/1024 (0%)]	Loss: 0.006435
********* Training the Unet Model **************
Train Epoch: 289 [0/1024 (0%)]	Loss: 0.006409
********* Training the Unet Model **************
Train Epoch: 290 [0/1024 (0%)]	Loss: 0.006378
********* Training the Unet Model **************
Train Epoch: 291 [0/1024 (0%)]	Loss: 0.006354
********* Training the Unet Model **************
Train Epoch: 292 [0/1024 (0%)]	Loss: 0.006327
********* Training the Unet Model **************
Train Epoch: 293 [0/1024 (0%)]	Loss: 0.006296
********* Training the Unet Model **************
Train Epoch: 294 [0/1024 (0%)]	Loss: 0.006275
********* Training the Unet Model **************
Train Epoch: 295 [0/1024 (0%)]	Loss: 0.006242
********* Training the Unet Model **************
Train Epoch: 296 [0/1024 (0%)]	Loss: 0.006219
********* Training the Unet Model **************
Train Epoch: 297 [0/1024 (0%)]	Loss: 0.006194
****

Train Epoch: 385 [0/1024 (0%)]	Loss: 0.004595
********* Training the Unet Model **************
Train Epoch: 386 [0/1024 (0%)]	Loss: 0.004579
********* Training the Unet Model **************
Train Epoch: 387 [0/1024 (0%)]	Loss: 0.004563
********* Training the Unet Model **************
Train Epoch: 388 [0/1024 (0%)]	Loss: 0.004551
********* Training the Unet Model **************
Train Epoch: 389 [0/1024 (0%)]	Loss: 0.004536
********* Training the Unet Model **************
Train Epoch: 390 [0/1024 (0%)]	Loss: 0.004521
********* Training the Unet Model **************
Train Epoch: 391 [0/1024 (0%)]	Loss: 0.004509
********* Training the Unet Model **************
Train Epoch: 392 [0/1024 (0%)]	Loss: 0.004495
********* Training the Unet Model **************
Train Epoch: 393 [0/1024 (0%)]	Loss: 0.004481
********* Training the Unet Model **************
Train Epoch: 394 [0/1024 (0%)]	Loss: 0.004467
********* Training the Unet Model **************
Train Epoch: 395 [0/1024 (0%)]	Loss: 0.004453
****

Train Epoch: 485 [0/1024 (0%)]	Loss: 0.003636
********* Training the Unet Model **************
Train Epoch: 486 [0/1024 (0%)]	Loss: 0.003628
********* Training the Unet Model **************
Train Epoch: 487 [0/1024 (0%)]	Loss: 0.003620
********* Training the Unet Model **************
Train Epoch: 488 [0/1024 (0%)]	Loss: 0.003613
********* Training the Unet Model **************
Train Epoch: 489 [0/1024 (0%)]	Loss: 0.003605
********* Training the Unet Model **************
Train Epoch: 490 [0/1024 (0%)]	Loss: 0.003598
********* Training the Unet Model **************
Train Epoch: 491 [0/1024 (0%)]	Loss: 0.003592
********* Training the Unet Model **************
Train Epoch: 492 [0/1024 (0%)]	Loss: 0.003591
********* Training the Unet Model **************
Train Epoch: 493 [0/1024 (0%)]	Loss: 0.003590
********* Training the Unet Model **************
Train Epoch: 494 [0/1024 (0%)]	Loss: 0.003586
********* Training the Unet Model **************
Train Epoch: 495 [0/1024 (0%)]	Loss: 0.003578
****

In [9]:
def plot_grid(fig, plot_input, output, actual_output, row_no):
        grid = ImageGrid(fig, 141, nrows_ncols=(row_no, 4), axes_pad=0.05, label_mode="1")
        for i in range(row_no):
                for j in range(3):
                        if(j == 0):
                                grid[i*4+j].imshow(np.transpose(plot_input[i], (1, 2, 0)), interpolation="nearest")
                        if(j == 1):
                                grid[i*4+j].imshow(np.transpose(output[i][0].detach().cpu().numpy(), (0, 1)), interpolation="nearest")
                        if(j == 2):
                                grid[i*4+j].imshow(np.transpose(actual_output[i][0].detach().cpu().numpy(), (0, 1)), interpolation="nearest")

output = model(first_rgb)

F = plt.figure(1, (30, 60))
F.subplots_adjust(left=0.05, right=0.95)
plot_grid(F, first_rgb, first_depth, output, 1)
plt.savefig("plots/small_unet_result.jpg")
plt.show()